In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import models
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
import os
import time
from PIL import Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_dataset = torchvision.datasets.ImageFolder(root='dataset_horse_vs_panda/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=40, shuffle=False)

In [ ]:
def test(model, test_loader, device, model_name):
    writer = SummaryWriter(f'runs/images/{model_name}')
    model.eval()

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            inputs = (inputs - inputs.min()) / (inputs.max() - inputs.min())
            img_grid = make_grid(inputs, nrow=8, normalize=True, scale_each=True)
            img_grid_np = img_grid.permute(1, 2, 0).cpu().numpy()
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.imshow(img_grid_np)
            ax.axis('off')
            batch_size = inputs.size(0)
            rows, cols = (batch_size // 8) + (batch_size % 8 > 0), 8  
            cell_height = img_grid_np.shape[0] / rows
            cell_width = img_grid_np.shape[1] / cols

            for idx in range(batch_size):
                row, col = divmod(idx, 8)
                text_x = col * cell_width + cell_width / 2
                text_y = row * cell_height + 20  

                ax.text(
                    text_x, text_y,
                    f"GT: {labels[idx].item()} | Pred: {predicted[idx].item()}",
                    fontsize=8, color="white", ha="center", va="top", bbox=dict(facecolor='black', alpha=0.6)
                )

            writer.add_figure(f'Test Batch {batch_idx}', fig, global_step=batch_idx)
            plt.close(fig)  

    writer.close()


In [28]:
class vgg_1_block(nn.Module):
    def __init__(self):
        super(vgg_1_block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 112 * 112, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

vgg1b = vgg_1_block()

In [36]:
vgg1b.load_state_dict(torch.load('vgg1b.pt', map_location=torch.device('cpu')))
test(vgg1b, test_loader, 'cpu', 'vgg1b')

C:\Users\Harshil Shah\AppData\Local\Temp\ipykernel_18588\702780820.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg1b.load_state_dict(torch.load('vgg1b.pt', map_locat

In [14]:
class vgg_3_block(nn.Module):
    def __init__(self):
        super(vgg_3_block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

vgg3b = vgg_3_block()
vgg3b.load_state_dict(torch.load('vgg3b.pt', map_location=torch.device('cpu')))


C:\Users\Harshil Shah\AppData\Local\Temp\ipykernel_18588\1801991783.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg3b.load_state_dict(torch.load('vgg3b.pt', map_loc

<All keys matched successfully>

In [21]:
test(vgg3b, test_loader, 'cpu', model_name='vgg3b')

In [23]:
vgg3b_aug = vgg_3_block()
vgg3b_aug.load_state_dict(torch.load('vgg3b_aug.pt', map_location=torch.device('cpu')))
test(vgg3b_aug, test_loader, 'cpu', model_name='vgg3b_aug')

C:\Users\Harshil Shah\AppData\Local\Temp\ipykernel_18588\3537438056.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg3b_aug.load_state_dict(torch.load('vgg3b_aug.pt', 

In [25]:
vgg16_all = models.vgg16(pretrained=True)
vgg16_all.classifier[6] = nn.Linear(4096, 2)
vgg16_all.load_state_dict(torch.load('vgg16_all.pt', map_location=torch.device('cpu')))
test(vgg16_all, test_loader, 'cpu', model_name='vgg16_all')

C:\Users\Harshil Shah\AppData\Local\Temp\ipykernel_18588\4108360330.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg16_all.load_state_dict(torch.load('vgg16_all.pt', 

In [27]:
vgg16_final = models.vgg16(pretrained=True)
vgg16_final.classifier[6] = nn.Linear(4096, 2)
vgg16_final.load_state_dict(torch.load('vgg16_final.pt', map_location=torch.device('cpu')))
test(vgg16_final, test_loader, 'cpu', model_name='vgg16_final')

C:\Users\Harshil Shah\AppData\Local\Temp\ipykernel_18588\4213100580.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg16_final.load_state_dict(torch.load('vgg16_final.p